In [ ]:
import syft as sy
import torch as th
import nest_asyncio
from syft.grid.webrtc_duet import Duet
from syft.grid.connections.http_connection import HTTPConnection
from syft.core.node.network.client import NetworkClient
from syft.core.io.route import SoloRoute
from nacl.signing import SigningKey

nest_asyncio.apply()
sy.VERBOSE = False

## Create a Signaling Client
This resource can be used / developed by different nodes (network, domain, device, vm) using different types of connection protocols (http, websocket, etc.)
Thus, we abstracted the concept of signaling server, in order to make this process as generic as possible.

In [ ]:
def create_signaling_client(url,conn_type, client_type):
    # Load an Signing Key instance
    signing_key = SigningKey.generate()
    verify_key = signing_key.verify_key
    
    # Use Signaling Server metadata
    # to build client route
    conn = conn_type(url=url)
    (
        spec_location,
        name,
        client_id,
    ) = client_type.deserialize_client_metadata_from_node(
            metadata=conn._get_metadata()
    )
    
    # Create a new Solo Route using the selected connection type
    route = SoloRoute(destination=spec_location, connection=conn)
    
    # Create a new signaling client using the selected client type
    signaling_client = client_type(
            network = spec_location,
            name=name,
            routes=[route],
            signing_key=signing_key,
            verify_key=verify_key)
    
    return signaling_client

In [ ]:
# In this example, we'll be using a Network Client type
# that uses HTTP connection to send its messages
signaling_client = create_signaling_client(
    url="http://localhost:5000",
    conn_type=HTTPConnection,
    client_type=NetworkClient)

## Create Our Domain
Now, we need to create our domain which will work as a server, computing remote requests.

In [ ]:
my_domain = sy.Domain(name="Maria")

## Define Target's Domain/Address
PS: We don't need to create a domain to our target peer (just its address would be enough).

In [ ]:
target = sy.Domain(name="Bob")

## Start Duet Process
Now, we can start our ICE protocol, if everything goes well, at the end of this process we'll be connected
with the other side and ready for send/receive messages.

**PS: It is important to pay attention to the "offer" parameter,if this parameter is the same for both peers, communication will not be established.**

In [ ]:
duet = Duet(node=my_domain,
            address=target.address,
            signaling_client=signaling_client,
            offer=False)

## Send/Receive Requests/Responses
The great advantage of a full-duplex channel is: You can act as a server and as a client simultaneously.

In [ ]:
print("Send/Request remote operations")
x = th.tensor([1,2,3,4,5,6])
p_x = x.send(duet)
for i in range(10):
    p_x = p_x + p_x
print("My Result: ", p_x.get())

In [ ]:
duet.close()